In [18]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [19]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002

n_hidden = 256
n_input = 28*28
n_noise = 128

In [20]:
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])


In [21]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [22]:
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))

D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [23]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1)+G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2)+G_b2)
    return output

In [24]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1)+D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2)+D_b2)
    return output

In [25]:
G = generator(Z)
D_fake = discriminator(G)
D_real = discriminator(X)

In [26]:
loss_D = tf.reduce_mean(tf.log(D_real)+ tf.log(1-D_fake))

In [27]:
loss_G = tf.reduce_mean(tf.log(D_fake))

In [28]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [29]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, 
                                                         var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,
                                                        var_list=G_var_list)


In [33]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [34]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0,0

In [37]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        _, loss_val_D = sess.run([train_D, loss_D],
                                feed_dict={X:batch_xs, Z:noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                feed_dict={Z:noise})
    print('Epoch:', '%04d' % epoch,
            'D loss: {:.4}'.format(loss_val_D),
            'G loss: {:.4}'.format(loss_val_G))
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise) # 노이즈 생성.
        samples = sess.run(G, feed_dict={Z: noise})
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
        plt.savefig('./samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
            
print('최적화 완료!')

Epoch: 0000 D loss: -0.7613 G loss: -1.688
Epoch: 0001 D loss: -0.6933 G loss: -1.909
Epoch: 0002 D loss: -0.6391 G loss: -1.835
Epoch: 0003 D loss: -0.6474 G loss: -2.19
Epoch: 0004 D loss: -0.7445 G loss: -1.643
Epoch: 0005 D loss: -0.7142 G loss: -1.912
Epoch: 0006 D loss: -0.6734 G loss: -1.884
Epoch: 0007 D loss: -0.7538 G loss: -1.961
Epoch: 0008 D loss: -0.6045 G loss: -1.83
Epoch: 0009 D loss: -0.7499 G loss: -1.638
Epoch: 0010 D loss: -0.6826 G loss: -2.04
Epoch: 0011 D loss: -0.7206 G loss: -1.88
Epoch: 0012 D loss: -0.7861 G loss: -1.853
Epoch: 0013 D loss: -0.7212 G loss: -2.024
Epoch: 0014 D loss: -0.6131 G loss: -2.22
Epoch: 0015 D loss: -0.5598 G loss: -2.009
Epoch: 0016 D loss: -0.9221 G loss: -2.002
Epoch: 0017 D loss: -0.7758 G loss: -1.833
Epoch: 0018 D loss: -0.6396 G loss: -2.108
Epoch: 0019 D loss: -0.7539 G loss: -1.923
Epoch: 0020 D loss: -0.7533 G loss: -1.77
Epoch: 0021 D loss: -0.6995 G loss: -1.983
Epoch: 0022 D loss: -0.7772 G loss: -1.989
Epoch: 0023 D los

C:\Python\Anaconda3\lib\site-packages\matplotlib\image.py:397: UserWarning: Warning: converting a masked element to nan.
  dv = (np.float64(self.norm.vmax) -
C:\Python\Anaconda3\lib\site-packages\matplotlib\image.py:398: UserWarning: Warning: converting a masked element to nan.
  np.float64(self.norm.vmin))
C:\Python\Anaconda3\lib\site-packages\matplotlib\image.py:405: UserWarning: Warning: converting a masked element to nan.
  a_min = np.float64(newmin)
C:\Python\Anaconda3\lib\site-packages\matplotlib\image.py:410: UserWarning: Warning: converting a masked element to nan.
  a_max = np.float64(newmax)
C:\Python\Anaconda3\lib\site-packages\matplotlib\colors.py:885: UserWarning: Warning: converting a masked element to nan.
  dtype = np.min_scalar_type(value)
C:\Python\Anaconda3\lib\site-packages\numpy\ma\core.py:713: UserWarning: Warning: converting a masked element to nan.
  data = np.array(a, copy=False, subok=subok)


Epoch: 0090 D loss: nan G loss: nan
Epoch: 0091 D loss: nan G loss: nan
Epoch: 0092 D loss: nan G loss: nan
Epoch: 0093 D loss: nan G loss: nan
Epoch: 0094 D loss: nan G loss: nan
Epoch: 0095 D loss: nan G loss: nan
Epoch: 0096 D loss: nan G loss: nan
Epoch: 0097 D loss: nan G loss: nan
Epoch: 0098 D loss: nan G loss: nan
Epoch: 0099 D loss: nan G loss: nan
최적화 완료!
